In [1]:
# Local Import
import numpy as np
import pandas as pd
import pickle

In [2]:
p_df1 = pd.read_csv('Alaska2000_5PUMS_Household.csv')
p_df2 = pd.read_csv('Alaska2000_5PUMS_Person.csv')
df = pd.merge(p_df2, p_df1, on='serial_no')
df = df.fillna({'RENT':0, 'VALUE':0})
df = df.rename(columns={'SUBSAMPL': 'Cell'})
rent = [df.RENT[i] / 4.35 * 1000 for i in range(len(df.RENT))]
renta = np.asarray(rent)

In [3]:
renta[0]=0
renta[np.logical_and(np.greater_equal(renta, 1), np.less(renta, 10000))]=1
renta[np.logical_and(np.greater_equal(renta, 10000), np.less(renta, 14999))]=2
renta[np.logical_and(np.greater_equal(renta, 15000), np.less(renta, 19999))]=3
renta[np.logical_and(np.greater_equal(renta, 20000), np.less(renta, 24999))]=4
renta[np.logical_and(np.greater_equal(renta, 25000), np.less(renta, 29999))]=5
renta[np.logical_and(np.greater_equal(renta, 30000), np.less(renta, 34999))]=6
renta[np.logical_and(np.greater_equal(renta, 35000), np.less(renta, 39999))]=7
renta[np.logical_and(np.greater_equal(renta, 40000), np.less(renta, 49999))]=8
renta[np.logical_and(np.greater_equal(renta, 50000), np.less(renta, 59999))]=9
renta[np.logical_and(np.greater_equal(renta, 60000), np.less(renta, 69999))]=10
renta[np.logical_and(np.greater_equal(renta, 70000), np.less(renta, 79999))]=11
renta[np.logical_and(np.greater_equal(renta, 80000), np.less(renta, 89999))]=12
renta[np.logical_and(np.greater_equal(renta, 90000), np.less(renta, 99999))]=13
renta[np.logical_and(np.greater_equal(renta, 100000), np.less(renta, 124999))]=14
renta[np.logical_and(np.greater_equal(renta, 125000), np.less(renta, 149999))]=15
renta[np.logical_and(np.greater_equal(renta, 150000), np.less(renta, 174999))]=16
renta[np.logical_and(np.greater_equal(renta, 175000), np.less(renta, 199999))]=17
renta[np.logical_and(np.greater_equal(renta, 200000), np.less(renta, 249999))]=18
renta[np.logical_and(np.greater_equal(renta, 250000), np.less(renta, 299999))]=19
renta[np.logical_and(np.greater_equal(renta, 300000), np.less(renta, 399999))]=20
renta[np.logical_and(np.greater_equal(renta, 400000), np.less(renta, 499999))]=21
renta[np.logical_and(np.greater_equal(renta, 500000), np.less(renta, 749999))]=22
renta[np.logical_and(np.greater_equal(renta, 750000), np.less(renta, 999999))]=23
renta[np.greater_equal(renta, 100000)]=24
df.RENT = renta
df.VALUE = (df.RENT + df.VALUE) / 24
df.age = df.age / max(df.age)
df.HINC = df.HINC / max(df.HINC)
df = df.dropna(subset=['YRBUILT'])
df.YRBUILT = df.YRBUILT / max(df.YRBUILT)
df = df[df.TENURE < 4]
df.RENT[df.RENT > 0] = True
df.RENT[df.RENT == 0] = False

In [4]:
df1 = df.drop(['SEX', 'INCTOT', 'RACE3', 'TENURE'], axis=1)
# Regression Value v Household income
df2 = df1.drop_duplicates('serial_no')
df3 = df2.drop(['age', 'YRBUILT', 'RENT', 'serial_no'], axis=1)
df3 = df3.reindex(columns=['Cell', 'VALUE', 'HINC', 'PUMA5'])
# Regression Yrbuilt v Age
df4 = df1.drop(['VALUE', 'HINC', 'RENT', 'serial_no'], axis=1)
df4 = df4.reindex(columns=['Cell', 'YRBUILT', 'age', 'PUMA5'])

In [5]:
ValHInc = df3.groupby('PUMA5')
YrBAge = df4.groupby('PUMA5')

In [143]:
# R1
A101 = df3[df3.PUMA5 == 101]
A102 = df3[df3.PUMA5 == 102]
A200 = df3[df3.PUMA5 == 200]
A300 = df3[df3.PUMA5 == 300]
A400 = df3[df3.PUMA5 == 400]

# R2
B101 = df4[df4.PUMA5 == 101]
B102 = df4[df4.PUMA5 == 102]
B200 = df4[df4.PUMA5 == 200]
B300 = df4[df4.PUMA5 == 300]
B400 = df4[df4.PUMA5 == 400]

In [144]:
class Wrangle:
    ''' Data Wrangling Class for Test Data '''
    def __init__(self, data, aggregate=False):
        # Numpy-nize Census Tract
        if 'PUMA5' in data:
            data = data.drop(['PUMA5'], axis=1)
        if data.keys()[0] != 'Cell':
            data = data[data.columns[::-1]]
        self.Key = data.keys()
        if aggregate:
            Y = np.asarray(data.loc[:, self.Key[1]])
            X = np.asarray(data.loc[:, self.Key[2]])
            self.N = len(X)
        else:
            if len(self.Key) > 3:
                None
            else:
                Cell = np.unique(np.array(data[self.Key[0]])).astype(int)
                Y = [np.asarray(data.loc[data.loc[:, self.Key[0]]== i, self.Key[1]]) for i in Cell]
                X = [np.asarray(data.loc[data.loc[:, self.Key[0]]== i, self.Key[2]]) for i in Cell]
                self.N = np.asarray([len(i) for i in X])
        self.X = X
        self.Y = Y 
        self.v = len(self.Key) - 1
                
    def __call__(self):
        return self.X, self.Y, self.N, self.v

In [149]:
VH101 = Wrangle(A101)
VH102 = Wrangle(A102)
VH200 = Wrangle(A200)
VH300 = Wrangle(A300)
VH400 = Wrangle(A400)

YA101 = Wrangle(B101)
YA102 = Wrangle(B102)
YA200 = Wrangle(B200)
YA300 = Wrangle(B300)
YA400 = Wrangle(B400)

In [9]:
%cd /home/nbuser/library/DP/Data/

/home/nbuser/library/DP/Data


In [11]:
with open('Alaska_VH.pickle', 'wb') as handle:
    pickle.dump((ValHInc), handle, pickle.HIGHEST_PROTOCOL)
with open('Alaska_YA.pickle', 'wb') as handle:
    pickle.dump((YrBAge), handle, pickle.HIGHEST_PROTOCOL)